<a href="https://colab.research.google.com/github/Prashuk2/Finetuning-FinBERT-for-Sentiment-Analysis/blob/main/Minor_Data_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data scrapping


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
base_url = "https://www.livemint.com/market/stock-market-news/page-{}"
headlines = []
for page in range(1, 110):
    url = base_url.format(page)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "lxml")
        articles = soup.find_all("div", class_="headlineSec")

        for article in articles:
            title_tag = article.find("h2").find("a")
            if title_tag:
                headlines.append(title_tag.text.strip())

        print(f"Page {page} scraped successfully!")
    else:
        print(f"Failed to retrieve page {page}")


df = pd.DataFrame({"Headline": headlines})


df.to_csv("livemint_stock_news.csv", index=False)

print("Scraping completed and data saved!")

Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!
Page 15 scraped successfully!
Page 16 scraped successfully!
Page 17 scraped successfully!
Page 18 scraped successfully!
Page 19 scraped successfully!
Page 20 scraped successfully!
Page 21 scraped successfully!
Page 22 scraped successfully!
Page 23 scraped successfully!
Page 24 scraped successfully!
Page 25 scraped successfully!
Page 26 scraped successfully!
Page 27 scraped successfully!
Page 28 scraped successfully!
Page 29 scraped successfully!
Page 30 scraped successfully!
Page 31 scraped successfully!
Page 32 scraped successfully!
Page 33 scraped successfully!
Page 34 scraped suc

In [ ]:
df

,Headline
0,"Tata Motors completes ₹2,000 crore fundraise m..."
1,How should investors navigate current stock ma...
2,"Sensex, Nifty 50 rise defying weak global cues..."
3,Dilip Buildcon shares surge 6% after securing ...
4,"Nifty 50, Sensex resume bull run after 1-day g..."
...,...
2181,Capital goods stocks rebound after 2-day slide...
2182,Titan Q3 Results: Profit dips 5% YoY to ₹990 c...
2183,JK Tyre Q3 Results: Net profit tanks 77% YoY t...
2184,"Kalyan Jewellers stock up 15% today, rallies 2..."


In [ ]:
import pandas as pd
import numpy as np

duplicate_count = df.duplicated().sum()
print("Total duplicate rows:", duplicate_count)




Total duplicate rows: 85


In [ ]:
df = df.drop_duplicates()


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.isnull().sum()

,0
Headline,0
sentiment_score,0
sentiment,0


In [ ]:
df.to_csv("data.csv", index=False)

Data from newsapi


In [ ]:
import requests
import pandas as pd
url = "https://newsapi.org/v2/everything"
params = {
    "q": "indian stock market",
    "language": "en",
    "sortBy": "publishedAt",
    "apiKey": "2e6595444e4b4386943e5d76c44486ca"
}
response = requests.get(url, params=params)
if response.status_code == 200:

    data = response.json()
    articles = data['articles']
    titles = []
    urls = []
    contents = []
    for article in articles:
        titles.append(article['title'])
        urls.append(article['url'])
        contents.append(article['content'] or "")


    df = pd.DataFrame({
        "Title": titles,
        "URL": urls,
        "Content": contents
    })
    print(df)
else:
    print(f"Failed to fetch news. Status code: {response.status_code}")
    print(response.json())

                                                Title  \
0   Gainers & Losers: HAL, Siemens among 6 stocks ...   
1   These 9 midcap stocks plunge 50%+ from their p...   
2   This brokerage spots a buying opportunity in L...   
3   Why stock market is falling today: 5 key facto...   
4   America's pain, India's gain? US recession can...   
..                                                ...   
94  Why the stock market is up today: 4 key factor...   
95  Auto sector outlook positive; two-wheelers off...   
96  Diageo plc (LON:DGE) Receives GBX 2,876.67 Con...   
97  Ashmore’s Medeiros says “Everybody loves India...   
98  Sensex Jumps Over 550 Points In Early Trade As...   

                                                  URL  \
0   https://m.economictimes.com/markets/stocks/new...   
1   https://m.economictimes.com/markets/stocks/new...   
2   https://economictimes.indiatimes.com/markets/s...   
3   https://economictimes.indiatimes.com/markets/s...   
4   https://economictimes.indi

In [ ]:
!pip install pandas nltk


In [ ]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk


nltk.download('vader_lexicon')


sia = SentimentIntensityAnalyzer()
df = pd.read_csv("data.csv")
print("Columns in CSV:", df.columns)
if "Headline" not in df.columns:
    raise ValueError("Column 'Headline' not found in data.csv. Check column names!")
df["Headline"] = df["Headline"].fillna("")
df["sentiment_score"] = df["Headline"].apply(lambda x: sia.polarity_scores(str(x))["compound"])
df["sentiment"] = df["sentiment_score"].apply(lambda x: "positive" if x > 0.05 else ("negative" if x < -0.05 else "neutral"))
df.to_csv("data_with_sentiment.csv", index=False)
print(df[["Headline", "sentiment_score", "sentiment"]].head())


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Columns in CSV: Index(['Headline'], dtype='object')
                                            Headline  sentiment_score  \
0  Tata Motors completes ₹2,000 crore fundraise m...           0.0000   
1  How should investors navigate current stock ma...           0.0000   
2  Sensex, Nifty 50 rise defying weak global cues...          -0.0516   
3  Dilip Buildcon shares surge 6% after securing ...           0.5423   
4  Nifty 50, Sensex resume bull run after 1-day g...           0.4019   

  sentiment  
0   neutral  
1   neutral  
2  negative  
3  positive  
4  positive  


Data from huggingface kdev


In [ ]:
import pandas as pd
from google.colab import files
from datasets import load_dataset

ds = load_dataset("kdave/Indian_Financial_News")

df = pd.DataFrame(ds['train'])


df = df.replace('\n', ' ', regex=True)


csv_filename = "indian_financial_news_full.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"All data saved to '{csv_filename}'")

files.download(csv_filename)

All this added to the one file

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving indian_financial_news1.csv to indian_financial_news1.csv
Saving balanced_sentiment_data.csv to balanced_sentiment_data.csv
Saving moneycontrol_news_balanced.csv to moneycontrol_news_balanced.csv


In [ ]:
import pandas as pd
import io
df1 = pd.read_csv(io.BytesIO(uploaded['indian_financial_news1.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded['balanced_sentiment_data.csv']))
df3 = pd.read_csv(io.BytesIO(uploaded['moneycontrol_news_balanced.csv']))

In [ ]:
sentiment_counts = df1['Sentiment'].value_counts()
print("Sentiment Counts:")
print(sentiment_counts)
positive_tweets = df1[df1['Sentiment'] == "Positive"]
print("\nPositive Tweets:")
print(positive_tweets)

Sentiment Counts:
Sentiment
Positive    8975
Negative    8957
Neutral     8951
Name: count, dtype: int64

Positive Tweets:
                                                 Content Sentiment
17908  Robust gains in shares of bank and financial h...  Positive
17909  live bse live  nse live Volume Todays L/H More...  Positive
17910  ICICI Lombard, a private sector non-life insur...  Positive
17911  Asian shares and Wall Street futures rallied o...  Positive
17912  MUMBAI: Reliance Industries reported a 9.8% ri...  Positive
...                                                  ...       ...
26878  Union Home Minister Amit Shah on Saturday comp...  Positive
26879  Saab Bags India’s First 100% FDI in Defence Pr...  Positive
26880  Crude oil futures gained to Rs 2,136 per barre...  Positive
26881  Bloomberg  Emerging markets struggling with hi...  Positive
26882  Two Indian cities on Thursday made it onto the...  Positive

[8975 rows x 2 columns]


In [ ]:
sentiment_counts = df1['Sentiment'].value_counts()
print("Sentiment Counts:")
print(sentiment_counts)
sentiment_counts = df2['Sentiment'].value_counts()
print("Sentiment Counts:")
print(sentiment_counts)
sentiment_counts = df3['Sentiment'].value_counts()
print("Sentiment Counts:")
print(sentiment_counts)

Sentiment Counts:
Sentiment
Positive    8975
Negative    8957
Neutral     8951
Name: count, dtype: int64
Sentiment Counts:
Sentiment
neutral     758
negative    700
positive    700
Name: count, dtype: int64
Sentiment Counts:
Sentiment
positive    83
negative    83
neutral     83
Name: count, dtype: int64


In [ ]:
import pandas as pd
df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
df['Sentiment'] = df['Sentiment'].str.lower().replace({
    'positive': 'Positive',
    'negative': 'Negative',
    'neutral': 'Neutral'
})
print(df)

                                                 Content Sentiment
0      US consumer spending dropped by a record in Ap...  Negative
1      State-run lenders require an urgent Rs 1.2 tri...  Negative
2      Apparel exporters on Wednesday urged the gover...  Negative
3      Asian shares battled to extend a global reboun...  Negative
4      After India’s sovereign credit rating fell to ...  Negative
...                                                  ...       ...
29285  The company, whose products range from insulin...   Neutral
29286  The company is yet to finalise the choice of l...   Neutral
29287  The decision to pause comes after Foxconn sent...   Neutral
29288  Given the tailwinds in the MF industry and the...   Neutral
29289  Summons have been sent to Adani's Shantivan Fa...   Neutral

[29290 rows x 2 columns]


In [ ]:
sentiment_counts = df['Sentiment'].value_counts()
print("Sentiment Counts:")
print(sentiment_counts)

Sentiment Counts:
Sentiment
Neutral     9792
Positive    9758
Negative    9740
Name: count, dtype: int64
